# Setup

In [ ]:
!pip install --no-index --no-deps --find-links /kaggle/input/requirements timm tf_slim
!pip install --no-index /kaggle/input/requirements/neuralgym-0.0.1.zip
!cp -r /kaggle/input/requirements/generative_inpainting /kaggle/generative_inpainting

# Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import random

In [ ]:
from tqdm import tqdm

import cv2
import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [ ]:
import inpaint

# Global

In [ ]:
IMG_SIZE = 512
PROJECT_FOLDER    = '/kaggle/input/creation-dataset-512x512/'
TRAIN_DATA_FOLDER = '/kaggle/input/creation-dataset-512x512/train_images/'
TEST_DATA_FOLDER  = '/kaggle/input/hotel-id-to-combat-human-trafficking-2022-fgvc9/test_images/'

In [ ]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
os.environ['PYTHONHASHSEED'] = str(0)
torch.backends.cudnn.deterministic = True

# Transformations

In [ ]:
import albumentations as A
import albumentations.pytorch as APT

In [ ]:
class RedSquare(A.DualTransform):
    def apply(self, img, **params):
        h, w, _ = img.shape
        width  = random.randint(int(w / 6), int(w / 4))
        height = random.randint(int(h / 6), int(h / 4))
        left   = random.randint(0, w - width)
        bot    = random.randint(0, h - height)
        img[bot:bot+height, left:left+width] = [0, 0, 255]
        return img

    def apply_to_bbox(self, bbox, **params):
        raise NotImplementedError()

    def apply_to_keypoint(self, keypoint, **params):
        raise NotImplementedError()

    def get_transform_init_args_names(self):
        return ()

In [ ]:
base_transform = A.Compose([
    A.ToFloat(),
    A.HorizontalFlip(p=0.5),
    APT.transforms.ToTensorV2(),
])

test_transform = A.Compose([
    A.ToFloat(),
    #RedSquare(p=1.0),
    APT.transforms.ToTensorV2(),
])

# Test Inpaint

In [ ]:
import matplotlib.pyplot as plt

image_id = random.choice(os.listdir(TRAIN_DATA_FOLDER))
image    = cv2.imread(TRAIN_DATA_FOLDER + image_id)
image    = cv2.resize(image, (512, 512))
square   = RedSquare().apply(image.copy())
inp      = inpaint.inpaint(square.copy())

f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,8))
ax1.imshow(image[...,::-1])
ax2.imshow(square[...,::-1])
ax3.imshow(inp)

# Dataset

In [ ]:
def open_and_preprocess_image(image_path):
    im = cv2.imread(image_path)
    im = cv2.resize(im, (512, 512))
    return im

class HotelImageDataset:
    def __init__(self, data, transform, data_folder, is_test):
        self.data = data
        self.transform = transform
        self.data_folder = data_folder
        self.is_test = is_test

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        record = self.data.iloc[idx]
        image_path = self.data_folder + str(record['image_id'])
        
        image = open_and_preprocess_image(image_path)
        if self.is_test:
            image = inpaint.inpaint(image)
        
        image = np.array(image).astype(np.uint8)
        return self.transform(image=image)

# Model

In [ ]:
class EmbeddingNet(nn.Module):
    def __init__(self, n_classes, embed_size, backbone_name):
        super(EmbeddingNet, self).__init__()
        
        self.embed_size = embed_size
        self.backbone = timm.create_model(backbone_name, pretrained=False)
        in_features = self.backbone.get_classifier().in_features

        fc_name, _ = list(self.backbone.named_modules())[-1]
        if fc_name == 'classifier':
            self.backbone.classifier = nn.Identity()
        elif fc_name == 'head.fc':
            self.backbone.head.fc = nn.Identity()
        elif fc_name == 'fc':
            self.backbone.fc = nn.Identity()
        else:
            raise Exception('Unknown classifier layer: ' + fc_name)
        
        self.post = nn.Sequential(
            nn.utils.weight_norm(nn.Linear(in_features, self.embed_size*2), dim=None),
            nn.BatchNorm1d(self.embed_size*2),
            nn.Dropout(0.2),
            nn.utils.weight_norm(nn.Linear(self.embed_size*2, self.embed_size)),
        )

        self.classifier = nn.Sequential(
            nn.BatchNorm1d(self.embed_size),
            nn.Dropout(0.2),
            nn.Linear(self.embed_size, n_classes),
        )
        
        print(f'Model {backbone_name} EmbeddingNet - Features: {in_features}, Embeds: {self.embed_size}')
        
    def embed_and_classify(self, x):
        x = self.forward(x)
        return x, self.classifier(x)

    def forward(self, x):
        x = self.backbone(x)
        x = x.view(x.size(0), -1)
        return self.post(x)

## Models

In [ ]:
def get_model(model_type, backbone_name, n_classes, embed_size, checkpoint_path):
    model = EmbeddingNet(n_classes, embed_size, backbone_name)
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['model'])
    return model.to('cuda')

In [ ]:
model_array = [
    get_model('classification', 'efficientnet_b0', 3043, 4096,
              '/kaggle/input/classification-training/classification-model-latest.pt'),
]

# Model helper functions

In [ ]:
def get_embeds(loader, model, bar_desc):
    outputs_all = []
    model.eval()
    with torch.no_grad():
        t = tqdm(loader, desc=bar_desc)
        for i, sample in enumerate(t):
            input = sample['image'].to('cuda')
            output = model(input)
            outputs_all.extend(output.detach().cpu().numpy())            
            
    return outputs_all

# Train and evaluate

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def get_distances(input, base_embeds):
    distances = 1
    for i, model in enumerate(model_array):
        output = model(input)
        output = output.detach().cpu().numpy()
        model_base_embeds = base_embeds[i]
        output_distances = cosine_similarity(output, model_base_embeds)
        distances *= output_distances
    
    return distances

def predict(loader, base_df, base_embeds, bar_desc):
    with torch.no_grad():
        t = tqdm(loader, desc=bar_desc)
        for i, sample in enumerate(t):
            input = sample['image'].to('cuda')
            distances = get_distances(input, base_embeds)
            
            for dist in distances:
                df = base_df.copy()
                df['distance'] = dist
                df = df.sort_values(by=['distance', 'hotel_id'], ascending=False).reset_index(drop=True)
                yield df['hotel_id'].unique()[:5]

def find_closest_match(test_loader, base_loader):
    base_embeds = {}
    for i, model in enumerate(model_array):
        base_embeds[i] = get_embeds(base_loader, model, 'Generating embeds for train')
    return predict(test_loader, base_loader.dataset.data, base_embeds, 'Generating predictions')

## Prepare Data

In [ ]:
data_df = pd.read_csv(PROJECT_FOLDER + 'train_df.csv').drop(['Unnamed: 0'], axis=1)

In [ ]:
# Validate
tmp = data_df.copy()
data_df = data_df.sample(frac=0.9, random_state=0)
test_df = tmp[~tmp['image_id'].isin(data_df['image_id'])]
TEST_DATA_FOLDER = TRAIN_DATA_FOLDER

In [ ]:
# Submission
#test_df = pd.DataFrame(data={'image_id': os.listdir(TEST_DATA_FOLDER), 'hotel_id': ''}).sort_values(by='image_id')

In [ ]:
print(f'Base: {len(data_df)}, test: {len(test_df)}')
data_df.head()

In [ ]:
base_dataset = HotelImageDataset(data_df, base_transform, TRAIN_DATA_FOLDER, False)
base_loader  = DataLoader(base_dataset, num_workers=0, batch_size=8, shuffle=False)

test_dataset = HotelImageDataset(test_df, test_transform, TEST_DATA_FOLDER, False)
test_loader  = DataLoader(test_dataset, num_workers=0, batch_size=8, shuffle=False)

## Inference

In [ ]:
# Validate
preds = np.array(list(find_closest_match(test_loader, base_loader)))
test_df['hotel_id_pred'] = [str(list(l)).strip('[]').replace(',', '') for l in preds]

In [ ]:
acc_pos_1 = (preds[:, 0] == test_df['hotel_id']).mean()
acc_pos_2 = (preds[:, 1] == test_df['hotel_id']).mean()
acc_pos_3 = (preds[:, 2] == test_df['hotel_id']).mean()
acc_pos_4 = (preds[:, 3] == test_df['hotel_id']).mean()
acc_pos_5 = (preds[:, 4] == test_df['hotel_id']).mean()
score = acc_pos_1 + (1 / 2) * acc_pos_2 + (1 / 3) * acc_pos_3 + (1 / 4) * acc_pos_4 + (1 / 5) * acc_pos_5
print(f'Score: {score:0.4f}')

In [ ]:
# Submission
#preds = find_closest_match(test_loader, base_loader)
#test_df['hotel_id'] = [str(list(l)).strip('[]').replace(',', '') for l in preds]
#test_df.to_csv('submission.csv', index=False)

In [ ]:
test_df.head()